In [21]:
using Graphs, GraphIO, FileIO
using CSV, DataFrames

using GMT

include("./src/cubes.jl")
include("./src/network.jl")

add_properties (generic function with 1 method)

In [22]:
## Get Vrancea subregion

# Read data
path = "./data/"
region = "Vrancea"
filepath = path * region * ".csv"

df = CSV.read(filepath, DataFrame);
minlon_vrancea = minimum(df.Longitude)
maxlon_vrancea = maximum(df.Longitude)
minlat_vrancea = minimum(df.Latitude)
maxlat_vrancea = maximum(df.Latitude)

# Read data
path = "./data/"
region = "Romania"
filepath = path * region * ".csv"

mkpath("./gmt/$region")

df = CSV.read(filepath, DataFrame);

cell_size = 5.0
minimum_magnitude = 4

# graph_path = "./networks/$region/cell_size_$(cell_size)km/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).txt"

region = "Vrancea"
df = df[(df.Longitude .> minlon_vrancea) .& (df.Longitude .< maxlon_vrancea) .& 
        (df.Latitude .> minlat_vrancea) .& (df.Latitude .< maxlat_vrancea), :]

df_filtered = df[df.Magnitude .> minimum_magnitude,:] 

# Split into cubes
df_filtered, df_filtered_cubes = region_cube_split(df_filtered,cell_size=cell_size,energyRelease=true);

# Create network
MG = create_network(df_filtered, df_filtered_cubes)
connectivity = degree(MG);

# edgelist_array = Matrix(edgelist);
edgelist = collect(edges(MG)) |> DataFrame;

In [23]:
# # Read data
# path = "./data/"
# region = "Romania"
# filepath = path * region * ".csv"

# mkpath("./gmt/$region")

# df = CSV.read(filepath, DataFrame);

# cell_size = 5.0
# minimum_magnitude = 4

# # graph_path = "./networks/$region/cell_size_$(cell_size)km/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).txt"

# df_filtered = df[df.Magnitude .> minimum_magnitude,:] 

# # Split into cubes
# df_filtered, df_filtered_cubes = region_cube_split(df_filtered,cell_size=cell_size,energyRelease=true);
# # Create network
# MG = create_network(df_filtered, df_filtered_cubes)
# connectivity = degree(MG);
# # edgelist_array = Matrix(edgelist);
# edgelist = collect(edges(MG)) |> DataFrame;

In [24]:
# Get region's coordinates
min_lon = minimum(df_filtered_cubes.cubeLongitude) -0.1
max_lon = maximum(df_filtered_cubes.cubeLongitude) +0.1
min_lat = minimum(df_filtered_cubes.cubeLatitude) -0.1
max_lat = maximum(df_filtered_cubes.cubeLatitude) +0.1
min_dep = minimum(df_filtered_cubes.cubeDepth);
max_dep = maximum(df_filtered_cubes.cubeDepth);

# Create the map coordinates
map_coords = [min_lon,max_lon,min_lat,max_lat]
map_coords_depth = [min_lon,max_lon,min_lat,max_lat,-max_dep,-min_dep]

# Colormap for the region topography
C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
# Relief map of the region
relief_map = grdcut("@earth_relief_30s", region=map_coords);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 26.1219 to 26.1166666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 27.0874 to 27.0916666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 45.2625 to 45.2583333333
grdcut [WARNING]: (n - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: n reset from 46.0023 to 46.0083333333


In [38]:
marker_size = connectivity ./10

# control marker color by connectivity
C_markers = makecpt(cmap=:seis, range=(minimum(connectivity),maximum(connectivity)), inverse=true);
zcolor_control = connectivity;

perspective = (135,25)
quality = 100

100

In [58]:
# Basemap to define the axes
basemap(limits=map_coords_depth, proj=:merc, zsize=16, frame="SEnwZ1+b xafg yafg zaf+lDepth(km)", par=(FONT_LABEL="24p,Palatino-Roman",MAP_LABEL_OFFSET=0.8, FONT_ANNOT="20p"), view=perspective)

# basemap!(limits=map_coords_depth, proj=:merc, zsize=16,
#         frame=(axes=(:left_full,:bottom_full,:right_full,:top_full), annot=200, ticks=100, xlabel=:horizontal, ylabel=:vertical),
#         par=(FONT_ANNOT_PRIMARY=10, FONT_LABEL=16, MAP_ANNOT_ORTHO=:we), view=perspective)

# Edges, plotted manually
for i in range(1,nrow(edgelist))
    line_coords = DataFrame(lats = [df_filtered_cubes.cubeLatitude[edgelist.src[i]],df_filtered_cubes.cubeLatitude[edgelist.dst[i]]],
                    lons =[df_filtered_cubes.cubeLongitude[edgelist.src[i]],df_filtered_cubes.cubeLongitude[edgelist.dst[i]]],
                    deps= [df_filtered_cubes.cubeDepth[edgelist.src[i]],df_filtered_cubes.cubeDepth[edgelist.dst[i]]])

    plot3d!(line_coords.lons, line_coords.lats, -line_coords.deps, JZ="16c", proj=:merc, pen=(:thinner,:black), alpha=50, view=perspective)
end

# Nodes
scatter3!(df_filtered_cubes.cubeLongitude, df_filtered_cubes.cubeLatitude, -df_filtered_cubes.cubeDepth,
limits=map_coords_depth,frame="SEnwZ1+b",proj=:merc, marker=:cube,markersize=0.3, #markersize=marker_size,
cmap=C_markers, zcolor=connectivity, 
alpha=50, view=perspective)

# Colorbar
colorbar!(limits=map_coords, pos=(paper=(24,10.0), size=(8,0.5)), shade=0.4, xaxis=(annot=2,), frame=(xlabel="Degree",),par=(FONT_LABEL="24p,Palatino-Roman,black",MAP_LABEL_OFFSET=0.6,FONT_ANNOT="20p"),view=(180,90))

# Relief map
grdview!(relief_map, proj=:merc, surftype=(image=quality,), 
cmap=C_map, zsize=1.3, alpha=25 ,yshift=14.6, view=perspective,
savefig="./gmt/$region/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).png",)


In [32]:
using Geodesy

In [33]:
min_lon

26.2219

In [37]:
min_dep

76.7

In [38]:
x0 = LLA(min_lon, min_lat, -min_dep)
xf = LLA(max_lon, min_lat, -min_dep)
lat_dist_in_km = Geodesy.euclidean_distance(xf,x0) / 1000

84.8137099094292

In [24]:
using DelimitedFiles

## Motifs plot polygons

In [25]:
motif="Triangle"
network_target_path ="./networks/$region/cell_size_$(cell_size)km/"
motif_filename = "motif$(motif)_$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).csv"

"motifTriangle_romania_side_5km_minmag_4.csv"

In [26]:
motifs = readdlm(network_target_path * motif_filename, ',', Int64);

In [42]:
# Basemap to define the axes
basemap(limits=map_coords_depth, proj=:merc, zsize=6, frame="SEnwZ1+b xafg yafg zaf+lDepth(km)", view=(135,20))

# Edges, plotted manually
for i in range(1,nrow(edgelist))
    line = DataFrame(lats = [df_filtered_cubes.cubeLatitude[edgelist.src[i]],df_filtered_cubes.cubeLatitude[edgelist.dst[i]]],
                    lons =[df_filtered_cubes.cubeLongitude[edgelist.src[i]],df_filtered_cubes.cubeLongitude[edgelist.dst[i]]],
                    deps= [df_filtered_cubes.cubeDepth[edgelist.src[i]],df_filtered_cubes.cubeDepth[edgelist.dst[i]]])

    plot3d!(line.lons,line.lats,-line.deps, JZ="6c", proj=:merc, pen=(:thinner,:black), alpha=50, view=(135,20))
end

# Nodes
scatter3!(df_filtered_cubes.cubeLongitude, df_filtered_cubes.cubeLatitude, -df_filtered_cubes.cubeDepth,
limits=map_coords_depth, frame="SEnwZ1+b xafg yafg zaf",proj=:merc, marker=:cube,markersize=0.1, #markersize=marker_size,
cmap=C_markers, zcolor=connectivity, 
alpha=50, view=(135,20))

# Motifs
for i in range(1,size(motifs,1))
    motif = DataFrame(lats = [df_filtered_cubes.cubeLatitude[motifs[i,1]], df_filtered_cubes.cubeLatitude[motifs[i,2]], df_filtered_cubes.cubeLatitude[motifs[i,3]]],
                    lons =[df_filtered_cubes.cubeLongitude[motifs[i,1]], df_filtered_cubes.cubeLongitude[motifs[i,2]], df_filtered_cubes.cubeLongitude[motifs[i,3]]],
                    deps= [df_filtered_cubes.cubeDepth[motifs[i,1]], df_filtered_cubes.cubeDepth[motifs[i,2]], df_filtered_cubes.cubeDepth[motifs[i,3]]])
    plot3d!(motif.lons,motif.lats,-motif.deps, JZ="6c", proj=:merc, L=true, G=:red, alpha=50, view=(135,20))
end

# Colorbar
colorbar!(pos=(outcell_size=:BC, offset=(0,1.5)), shade=0.4, JZ="6c", xaxis=(annot=:auto,), frame=(xlabel="Degree",),par=(MAP_LABEL_OFFSET=0.8,))

# Relief map
grdview!(relief_map, proj=:merc, surftype=(image=2000,), 
cmap=C_map, zsize=0.05, alpha=10 ,yshift=5.6, view=(135,20),
savefig="./networks/$region/cell_size_$(cell_size)km/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).png", 
show=true)